# Geometric Intution + Rectangle Decomposable Bimodules 

In this notebook, we visit an example where we explore the geometric interpretation of the block function in dimension 0.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist

import iblofunmatch.inter as ibfm
output_dir = "output" 
plots_dir = "plots/geom_interpretation"

import os 
os.makedirs(output_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

Create a pair $X \subseteq Y$ such that it leads to an interesting matching.

In [ ]:
Y = []
Y = [[0,0], [1,0], [3,0]]
indices_subset = [0,1,2]
for idx in range(1, 20):
    if idx ==10:
        continue
    angle = np.pi * (1 + idx/20)
    Y.append([1.5 + 1.5*np.cos(angle), 1.5*np.sin(angle)])
# Next, add further points 
Y += [[1+0.2*i, 0] for i in range(1,7)]
Y = np.array(Y)

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
X = Y[indices_subset]
ax.scatter(Y[:,0], Y[:,1], c="blue", zorder=1)
ax.scatter(X[:,0], X[:,1], c="red", zorder=2, marker="X", s=150)
ax.set_aspect("equal")
fig.tight_layout()
plt.savefig(f"{plots_dir}/points_example.png")

Now, we are ready to compute the block function $\textrm{PH}_0(X)\rightarrow \textrm{PH}_0(Y)$

In [ ]:
ibfm_output = ibfm.get_IBloFunMatch_subset(None, Y, indices_subset, output_dir, num_it=4, max_rad=-1, points=True, max_dim=1)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(5,2))
ibfm.plot_matching(ibfm_output, ax, fig, block_function=True, dim=0)
fig.tight_layout()
plt.savefig(f"{plots_dir}/block_funct_0_example.png")

In [ ]:
ibfm_output["X_barcode_0"].shape

Now, we plot the matching using a geometric intuition.

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(10,4))
for idx, pair_ab in enumerate([[0,24], [0,25],[1,24]]):
    a = ibfm_output["S_barcode_0"][:,1][pair_ab[0]]
    b = ibfm_output["X_barcode_0"][:,1][pair_ab[1]]
    ibfm.plot_geometric_matching(a, b, indices_subset, Y, ibfm_output, ax[idx], labelsize=6)
plt.tight_layout()
plt.savefig(f"{plots_dir}/matching_geometric_half_circle.png")

Next, we depict the matching using rectangles.

In [ ]:
import matplotlib as mpl

In [ ]:
def plot_bimodule_0_blofun(ibfm_output, ax, fig):
    S_ends = ibfm_output["S_barcode_0"][:,1]
    X_ends = ibfm_output["X_barcode_0"][:,1]
    blofun = ibfm_output["block_function_0"]
    unmatched = [i for i in range(len(X_ends)) if i not in blofun]
    hlim = np.max(S_ends)*1.6
    # Fill horizontal turquoise bands for unmatched intervals 
    for idx in unmatched:
        b = X_ends[idx]
        ax.add_patch(mpl.patches.Rectangle([0, 0], hlim,  b, facecolor="turquoise", edgecolor="none", alpha=0.2, zorder=0.5))
        ax.plot([0,hlim], [b,b], linewidth=1, color="gray", zorder=1)
    # end for 
     # Fill matched intervals with corresponding orange rectangles
    match_points = []
    for idx, idx_match in enumerate(blofun):
        a = S_ends[idx]
        b = X_ends[idx_match]
        match_points.append([a,b])
        ax.add_patch(mpl.patches.Rectangle([0, 0], a, b, facecolor="orange", edgecolor="none", alpha=0.4, zorder=0.6))
        ax.plot([0,a,a], [b,b,0], linewidth=1, color="gray", zorder=1)
    # for 
    match_points = np.array(match_points)
    ax.scatter(match_points[:,0], match_points[:,1], s=30, marker="o", facecolors="none", edgecolors="orange")
    # draw diagonal line for reference 
    ax.plot([0,hlim], [0, hlim], linestyle="dashed", linewidth=1, color="black")
    # Set limits 
    ax.set_xlim([0, hlim*0.9])
    ax.set_ylim([0, np.max(X_ends)*1.3])
    ax.set_aspect("equal")
    fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(6,2.5))
plot_bimodule_0_blofun(ibfm_output, ax, fig)
fig.tight_layout()
plt.savefig(f"{plots_dir}/bimodule_plot_blofun_0.png")

In [ ]:
ibfm_output["X_barcode_0"]